In [ ]:
%run "Подготовка данных.ipynb"

In [ ]:
def result(X, y):
    t = X.isnull().sum(axis = 0).max()
    y = y[t:]            
    X = X[t:] 
    X = X.reset_index(drop = True)
    y = y.reset_index(drop = True)
    X = sm.add_constant(X)
    n = round(len(X)*0.6)
    return X, y, n, t


In [ ]:
def match_func(index, lag, start, results, h):
    pattern = re.compile(var_name + index)
    match  = pattern.search(str(first_model1))
    if match:
        predictions_count = []
        start = results.fittedvalues.index[-1]
        for i in range(start, start+h+1):
            ### Назначаем текущее значение прогноза у его лагу.  
            X[var_name+index][i-lag+1] = results.predict(X[i:i+1])
            predictions_count.append(X[var_name+index][i-lag+1])
    return predictions_count

def get_predictions(h):
    try:
        predictions = match_func('-1', 1, results = results, h=h)
    except:
        try:
            predictions = match_func('-2', 2, results = results, h=h)
        except:
            try:
                predictions = match_func('-3', 3, results = results, h=h)
            except:
                predictions = []
                for i in range(start, start+h+1):
                     ## добавляем прогнозное наблюдение
                    predictions.append(results.predict(X[i:i+h+1]))
    return predictions

In [ ]:
prod_unit_ = pd.read_excel('data_categ_unit.xlsx', sheet_name = 'prod_yty').sort_values(by='dt',ascending = True).reset_index(drop=True)
neprod_unit_ = pd.read_excel('data_categ_unit.xlsx', sheet_name = 'neprod_yty').sort_values(by='dt',ascending = True).reset_index(drop=True)

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 100))
scaler = scaler.fit(prod_unit_.iloc[:,1:])
try1 = scaler.transform(prod_unit_.iloc[:,1:])
prod_unit = pd.DataFrame(try1, columns=prod_unit_.iloc[:,1:].columns)


scaler = scaler.fit(neprod_unit_.iloc[:,1:])
try1 = scaler.transform(neprod_unit_.iloc[:,1:])
neprod_unit = pd.DataFrame(try1, columns=neprod_unit_.iloc[:,1:].columns)

In [ ]:
for i in prod_unit.columns[1:]: 
        prod_unit[i+'-1'] = prod_unit[i].shift().reset_index(drop = True)
        prod_unit[i+'-2'] = prod_unit[i+'-1'].shift().reset_index(drop = True)
        prod_unit[i+'-3'] = prod_unit[i+'-2'].shift().reset_index(drop = True) 

In [ ]:
prod_unit.head(2)

In [ ]:
for i in neprod_unit.columns[1:]: 
        neprod_unit[i+'-1'] = neprod_unit[i].shift().reset_index(drop = True)
        neprod_unit[i+'-2'] = neprod_unit[i+'-1'].shift().reset_index(drop = True)
        neprod_unit[i+'-3'] = neprod_unit[i+'-2'].shift().reset_index(drop = True) 

In [ ]:
neprod_unit.head(2)

Выбираем таблицу для расчета результатов 

In [ ]:
table_unit = neprod_unit

In [ ]:
table_items = pd.DataFrame()

for key in range(1,len(table_unit.columns)):
    try:
        var_name = table_unit.columns[key]

        variables = [tab_2014['ipd'], tab_2014['ipd-1'], tab_2014['ipd-2'], tab_2014['ipd-3'],
                    tab_2014['oil'], tab_2014['oil-1'], tab_2014['oil-2'],tab_2014['oil-3'],
                    tab_2014['rvi'], tab_2014['rvi-1'], tab_2014['rvi-2'],tab_2014['rvi-3'],
                    tab_2014['real_rate'], tab_2014['real_rate-1'],tab_2014['real_rate-2'],tab_2014['real_rate-3'],
                    table_unit[var_name+'-1'], table_unit[var_name+'-2'], table_unit[var_name+'-3']]

        combs = []
        for i in range(1, 4):
            els = [list(x) for x in itertools.combinations(variables, i)]
            combs.extend(els) 

        inf = []
        for m in range(len(combs)):
            X = pd.DataFrame(combs[m]).T
            y = table_unit.filter([table_unit.columns[key]])
            X, y, n, t = result(X, y)
            model = sm.OLS(y[:n], X[:n])
            results = model.fit(cov_type='HC3')

            start = results.fittedvalues.index[-1]
            predictions = []
            for i in range(start, y.index[-1]):
                ## добавляем прогнозное наблюдение
                a = results.predict(X[i:i+1])
                predictions.append(a[i])
                model = sm.OLS(y[:i], X[:i])
                results = model.fit(cov_type='HC3')

            columns = X.columns
            r = results.rsquared_adj
            p = results.pvalues
            k=[]
            for i in range(0,len(p)):
                if p[i] < 0.05:
                    k.append(p.index[i])
            MAE = mean_absolute_percentage_error(y[n:][var_name], predictions)         
            inf.append([list(X.columns), k, r, MAE])

        ols_all = pd.DataFrame(inf, columns = ['Регрессоры', 'Значимые регрессоры', 'Скорректированный R^2', 'MAE'])
        p_values = []
        regressors = []
        for i in range(len(ols_all)):
            p_values.append(", ".join(ols_all['Значимые регрессоры'][i])) 
            regressors.append(", ".join(ols_all['Регрессоры'][i])) 
        ols_all['Значимые регрессоры'] = p_values
        ols_all['Регрессоры'] = regressors

        df = pd.DataFrame()
        for j in range(len(ols_all['Значимые регрессоры'])):
            if ', '.join(ols_all['Регрессоры'][j].split(', ')[1:]) in ols_all['Значимые регрессоры'][j]:
                df = df.append(ols_all[ols_all.index == j])
        models = df.sort_values('Скорректированный R^2', ascending=False).reset_index(drop=True)[:10]

        if table_unit.iloc[0,1] == neprod_unit.iloc[0,1]:
            goods = neprod_detrend ## Выбираем тренды к целевой переменной
        else: 
            goods = prod_detrend 

        combs = []
        for i in range(1, 3):
            els = [list(x) for x in itertools.combinations(goods, i)]
            combs.extend(els)

        s = pd.DataFrame()
        for g in range(len(combs)):
            inf = []
            for m in range(len(models)):
                splited = models['Регрессоры'][m].split(', ')
                X = tab_2014.filter(splited) 
                X_ = X.join(table_unit.filter(splited))
                X = X_.join(goods[combs[g]], how = 'left')
                y = table_unit.filter([table_unit.columns[key]])
                X, y, n, t = result(X, y)
                model = sm.OLS(y[:n], X[:n])
                results = model.fit(cov_type='HC3')
                start = results.fittedvalues.index[-1]
                predictions = []
                for i in range(start, y.index[-1]):
                    ## добавляем прогнозное наблюдение
                    a = results.predict(X[i:i+1])
                    predictions.append(a[i])
                    model = sm.OLS(y[:i], X[:i])
                    results = model.fit(cov_type='HC3')
                columns = X.columns
                p = results.pvalues
                k=[]
                for i in range(0,len(p)):
                    if p[i] < 0.05:
                        k.append(p.index[i])
                r = results.rsquared_adj
                MAE = mean_absolute_percentage_error(y[n:][var_name], predictions)
                inf.append([list(X.columns), k, r, MAE])    
            ols_all = pd.DataFrame(inf, columns = ['Регрессоры', 'Значимые регрессоры', 'Скорректированный R^2', 'MAE'])
            table = models.join(ols_all, how = 'left', rsuffix=' (тренды)')
            table['Разница MAE'] = table['MAE (тренды)'] - table['MAE']
            table['Разница R^2'] = table['Скорректированный R^2 (тренды)'] - table['Скорректированный R^2']
            table['Регрессант'] = table_unit.columns[key]
            table1 = table[(table['Разница MAE']<0) & (table['Разница R^2']>0)].reset_index(drop = True)
            for j in range(len(table1['Значимые регрессоры (тренды)'])):
                if combs[g][0] in table1['Значимые регрессоры (тренды)'][j]:
                    s = s.append(table1[table1.index == j])
        table_items = table_items.append(s)
    except:
        pass

In [ ]:
table_items['Регрессант'].unique()

In [ ]:
## Выбираем отдельную категорию 

s = table_items[table_items['Регрессант']=='Компьютеры'].reset_index(drop=True)
len(s)

In [ ]:
for i in range(len(s['Значимые регрессоры (тренды)'])):
    k=re.findall('Comp', str(s['Значимые регрессоры (тренды)'][i]))
    if len(k)>0:
        print(i)

In [ ]:
table_items.sort_values('Скорректированный R^2 (тренды)', ascending = False).reset_index(drop = True)

In [ ]:
if table_unit.iloc[0,1] == neprod_unit.iloc[0,1]:
    goods = neprod_detrend ## Выбираем тренды к целевой переменной
else: 
    goods = prod_detrend 

In [ ]:
DM = pd.DataFrame()
s = s.reset_index(drop = True)
for h in range(10):
    dm = []
    for j in range(len(s)):
        first_model1 = s.iloc[j,:]['Регрессоры'].split(', ')
        X_ = tab_2014.filter(first_model1)
        X = X_.join(table_unit.filter(first_model1))
        y = table_unit[s['Регрессант'][0]]
        X, y, n, t = result(X, y)
        model = sm.OLS(y[:n], X[:n])
        results = model.fit(cov_type='HC3')

        start = results.fittedvalues.index[-1]
        predictions1 = []
        for i in range(start, y.index[-1]-h):
            predictions = get_predictions(h)
            predictions1.append(predictions[0][start+h])
            start +=1 
            X_ = tab_2014.filter(first_model1)  
            X = X_.join(table_unit.filter(first_model1))
            y = table_unit[s['Регрессант'][0]]
            X, y, n, t = result(X, y)
            model = sm.OLS(y[:i], X[:i])
            results = model.fit(cov_type='HC3')

        first_model1 = s.iloc[j,:]['Регрессоры'].split(', ')
        X_star = tab_2014.filter(first_model1)
        X_ = X_star.join(table_unit.filter(first_model1))
        X = X_.join(goods[s.iloc[j,4][len(s.iloc[j,0].split(', ')):]])

        y = table_unit[s['Регрессант'][0]]
        X, y, n, t = result(X, y)
        model = sm.OLS(y[:n], X[:n])
        results = model.fit(cov_type='HC3')
        
        start = results.fittedvalues.index[-1]
        predictions2 = []
        for i in range(start, y.index[-1]-h):
            predictions = get_predictions(h)
            predictions2.append(predictions[0][start+h])
            start +=1 
            X_star = tab_2014.filter(first_model1)
            X_ = X_star.join(table_unit.filter(first_model1))
            X = X_.join(goods[s.iloc[j,4][len(s.iloc[j,0].split(', ')):]])
            y = table_unit[s['Регрессант'][0]]
            X, y, n, t = result(X, y)
            model = sm.OLS(y[:i], X[:i])
            results = model.fit(cov_type='HC3')
      

        dm.append(dm_test(y[n:][h:], predictions2, predictions1, h = h+1, crit = 'MSE')[1])
    
        
    df = pd.DataFrame()
    for i in range(len(dm)):
        if dm[i]<0.2:
            df = df.append(s.iloc[i,:])
    df['Шаг прогноза'] = h+1
    DM = DM.append(df)

In [ ]:
DM

In [ ]:
df1 = DM[['Регрессоры (тренды)', 'Шаг прогноза']]
df2 = df1.groupby(level=0)['Шаг прогноза'].apply(list).to_dict()
keys = list(df2.keys())
names_ = []
for i in s.index:
    if i in keys:
        names_.append(str(s['Регрессоры (тренды)'][i]))

print('Количество уникальных регрессий ', len(pd.DataFrame([names_, df2.values()]).T))

## Подсчет количества моделей с шагом прогноза от 1 до 10.
#########################################################

for i in range(1,11):
    print(len(DM[DM['Шаг прогноза'] == i]))

In [ ]:
## Список моделей для конкретного шага прогноза 
###############################################

temp = DM[DM['Шаг прогноза'] == 1].sort_values(['Разница R^2'], ascending = False)
temp = temp.reset_index(drop = True)
temp